# UTSA CS 3793: Assignment-2

**Ramos - Kendall - (yjk504)**






In [203]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Learning Objectives



*   Game Playing
*   Chess - Board Setup & Rules
*   Adversarial Search
*   AI - Random vs MinMax



## Description

This assignment is focused on **game playing** and creating a proper **AI for chess**.
In the following sections, you will complete a series of tasks to create a chess game board, rules for each chess piece, a Random AI and a MinMax AI that plays a game of chess for both players (white and black).

The base structure of the code is provided. You are supposed to write code for each of the functions. Comments are provided on what should be done. You **CANNOT** use a complete chess library and change the base code structure completely. However, you **CAN** change the code layout and name/format of the functions.


#Chess Board Setup & Rules

In this section, you will write code to import the necessary libraries and create:

1.   **ChessBoard** - This part will contain code to initialize the board, draw the board, get the board state and move piece.
2.   **ChessRules** - This part will contain code for the chess rules for each piece.



##Import Libraries

The code here will contain only **import** statements. A base list of the required libraries are already imported. You will most likely not need any other libraries, but if needed, add the import statements here. As mentioned before, you can not use any premade chess libraries.

In [204]:
# your code goes here
import string
import random
import os
import sys
import time
from IPython.display import clear_output
import numpy as np
import copy
import itertools
import math
import collections
import functools



##ChessBoard

(10 points)

Fill the code in the code structure provided below for the ChessBoard. The main use of this code block write functions to initialize the board, draw the board, get the board state and move piece. You can add any other functions if needed.


In [205]:
# Define constants for chess pieces
EMPTY = '.'
PAWN_WHITE = 'P'
PAWN_BLACK = 'p'
ROOK_WHITE = 'R'
ROOK_BLACK = 'r'
KNIGHT_WHITE = 'T'
KNIGHT_BLACK = 't'
BISHOP_WHITE = 'B'
BISHOP_BLACK = 'b'
QUEEN_WHITE = 'Q'
QUEEN_BLACK = 'q'
KING_WHITE = 'K'
KING_BLACK = 'k'

# Initialize the chess board
def ChessBoardSetup():
    board = [
        [ROOK_BLACK, KNIGHT_BLACK, BISHOP_BLACK, QUEEN_BLACK, KING_BLACK, BISHOP_BLACK, KNIGHT_BLACK, ROOK_BLACK],
        [PAWN_BLACK] * 8,
        [EMPTY] * 8,
        [EMPTY] * 8,
        [EMPTY] * 8,
        [EMPTY] * 8,
        [PAWN_WHITE] * 8,
        [ROOK_WHITE, KNIGHT_WHITE, BISHOP_WHITE, QUEEN_WHITE, KING_WHITE, BISHOP_WHITE, KNIGHT_WHITE, ROOK_WHITE]
    ]
    return board

# Draw the chess board
def DrawBoard(board):
    for row in board:
        print(' '.join(row))
    print()

# Move a chess piece
def MovePiece(board, from_square, to_square):
    from_row, from_col = from_square
    to_row, to_col = to_square

    piece = board[from_row][from_col]
    board[to_row][to_col] = piece
    board[from_row][from_col] = EMPTY


##ChessRules

(50 points)

Fill the code in the code structure provided below for ChessRules. The main use of the code block is to write functions to design the rules for movement of each piece on the board. This block will also contain the function to check if the current player is in check, check-mate. You can also have functions that can return the current player's pieces that have legal moves in the current board state.

Following are some **suggested** functions with the pseudocode provided. You can create/remove functions as needed.


In [206]:
# Constants for piece types
PAWN_WHITE = "WP"
PAWN_BLACK = "BP"
ROOK_WHITE = "WR"
ROOK_BLACK = "BR"
BISHOP_WHITE = "WB"
BISHOP_BLACK = "BB"
QUEEN_WHITE = "WQ"
QUEEN_BLACK = "BQ"
KNIGHT_WHITE = "WN"
KNIGHT_BLACK = "BN"
KING_WHITE = "WK"
KING_BLACK = "BK"
EMPTY = "--"

# Constants for board dimensions
BOARD_SIZE = 8

# Placeholder for the game board
board = [[EMPTY for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]

def IsMoveLegal(board, from_square, to_square):
    from_row, from_col = from_square
    to_row, to_col = to_square

    # Get the from-piece and to-piece
    from_piece = board[from_row][from_col]
    to_piece = board[to_row][to_col]

    # Check if from-square is the same as to-square
    if from_square == to_square:
        return False

    # Define a helper function to check if a square is within the board bounds
    def is_within_bounds(row, col):
        return 0 <= row < BOARD_SIZE and 0 <= col < BOARD_SIZE

    # Define a helper function to check if a square is empty
    def is_empty(row, col):
        return board[row][col] == EMPTY

    # Check for pawn moves
    if from_piece == PAWN_WHITE:
        # Pawn moves one step forward (or backward if white)
        if from_col == to_col and is_empty(to_row, to_col):
            if (from_row - to_row == 1) or (from_row == 6 and from_row - to_row == 2):
                return True
        # Pawn attacks diagonally
        elif abs(from_col - to_col) == 1 and from_row - to_row == 1 and to_piece != EMPTY:
            return True

    elif from_piece == PAWN_BLACK:
        # Pawn moves one step forward (or backward if black)
        if from_col == to_col and is_empty(to_row, to_col):
            if (to_row - from_row == 1) or (from_row == 1 and to_row - from_row == 2):
                return True
        # Pawn attacks diagonally
        elif abs(from_col - to_col) == 1 and to_row - from_row == 1 and to_piece != EMPTY:
            return True
            # Check for rook moves
    if from_piece == ROOK_WHITE or from_piece == ROOK_BLACK:
        # Rook moves horizontally or vertically
        if from_row == to_row or from_col == to_col:
            if IsClearPath(board, from_square, to_square):
                return True

    # Check for other piece types (rook, bishop, queen, knight, king)
    # Add similar checks following your specifications

    return False  # If none of the above conditions are met, the move is not legal

def GetListOfLegalMoves(board, from_square):
    from_row, from_col = from_square
    from_piece = board[from_row][from_col]

    legal_moves = []

    # Iterate over all squares on the board
    for to_row in range(BOARD_SIZE):
        for to_col in range(BOARD_SIZE):
            to_square = (to_row, to_col)

            # Check if the move is legal
            if IsMoveLegal(board, from_square, to_square):
                # Check if the move puts the player in check
                if not DoesMovePutPlayerInCheck(board, from_square, to_square):
                    legal_moves.append(to_square)

    return legal_moves

def GetPiecesWithLegalMoves(board, player):
    pieces_with_legal_moves = []

    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            from_square = (row, col)
            from_piece = board[row][col]

            # Check if the square contains a piece belonging to the current player
            if from_piece.startswith(player):
                legal_moves = GetListOfLegalMoves(board, from_square)

                # Check if there are any legal moves for the piece
                if legal_moves:
                    pieces_with_legal_moves.append(from_square)

    return pieces_with_legal_moves

def IsCheckmate(board, player):
    # Get all pieces with legal moves for the current player
    pieces_with_legal_moves = GetPiecesWithLegalMoves(board, player)

    # If there are no pieces with legal moves, it's checkmate
    return not bool(pieces_with_legal_moves)

def IsInCheck(board, player):
    # Find the given player's King's location
    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            if board[row][col] == f"K{player[1]}":
                king_square = (row, col)

    # Iterate over all squares on the board
    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            from_square = (row, col)
            from_piece = board[row][col]

            # Check if there is a piece at that location and that piece is of the enemy team
            if from_piece.startswith("B" if player == "W" else "W"):
                # Check if the move is legal for the enemy player to attack the king
                if IsMoveLegal(board, from_square, king_square):
                    return True

    return False

def IsClearPath(board, from_square, to_square):
    # Given the move (from-square and to-square)

    # If the from and to squares are only one square apart
    if abs(from_square[0] - to_square[0]) <= 1 and abs(from_square[1] - to_square[1]) <= 1:
        return True
    else:
        # Calculate row and column differences
        row_diff = to_square[0] - from_square[0]
        col_diff = to_square[1] - from_square[1]

        # Determine the direction of the move
        if row_diff < 0:
            row_dir = -1
        elif row_diff > 0:
            row_dir = 1
        else:
            row_dir = 0

        if col_diff < 0:
            col_dir = -1
        elif col_diff > 0:
            col_dir = 1
        else:
            col_dir = 0

        # Calculate the new-from-square
        new_from_row = from_square[0] + row_dir
        new_from_col = from_square[1] + col_dir
        new_from_square = (new_from_row, new_from_col)

        # Check if the path is clear by recursively checking each square along the path
        if board[new_from_row][new_from_col] != EMPTY:
            return False
        else:
            return IsClearPath(board, new_from_square, to_square)

def DoesMovePutPlayerInCheck(board, from_square, to_square):
    # Given the move (from-square and to-square)
    from_row, from_col = from_square
    to_row, to_col = to_square

    # Get the from-piece and to-piece
    from_piece = board[from_row][from_col]
    to_piece = board[to_row][to_col]

    # Temporarily make the move by changing the board
    board[to_row][to_col] = from_piece
    board[from_row][from_col] = EMPTY

    # Check if the current player is in check
    player = from_piece[0]
    is_in_check = IsInCheck(board, player)

    # Undo the temporary move
    board[from_row][from_col] = from_piece
    board[to_row][to_col] = to_piece

    return is_in_check


#Artificial Intelligence

In this section, you will write code for the Artificial Intelligence (AI) that will play a game of chess. You will write 2 types of AI:

1.   **RandomAI** - This part will contain code for moving a chess piece randomly.
2.   **MinMaxAI** - This part will contain code for moving a chess piece using the MinMax strategy discussed in the lecture.


##RandomAI

(10 points)

Complete the function below that will perform a random move for the given player. The function will return the move (from-piece and to-piece). You will most likely not need to write any other function, but you can, if needed.


In [207]:
def GetRandomMove(board, player):
    # Get a list of all pieces for the current player that have legal moves
    pieces_with_legal_moves = GetPiecesWithLegalMoves(board, player)

    # Check if there are any pieces with legal moves
    if not pieces_with_legal_moves:
        return None  # No legal moves available

    # Select a random piece from the list
    random_piece = random.choice(pieces_with_legal_moves)

    # Get a list of legal moves for the selected piece
    legal_moves = GetListOfLegalMoves(board, random_piece)

    # Select a random legal move for the piece
    random_move = random.choice(legal_moves)

    return (random_piece, random_move)

# Example usage:
# random_move = GetRandomMove(board, "W")
# where "W" is the current player (White)




##MinMaxAI

(50 points)

Complete the functions below that will perform a move for the given player using the MinMax AI strategy. One function will evaluate the board if a move is performed - give score for each of piece and calculate the score for the entire chess board. In the second function you will write actual code for the MinMax strategy and return the move (from-piece and to-piece). To get the allocated points, searching should be **2-ply (one Max and one Min)**. You will most likely not need to write any other function, but you can, if needed.

## Extra Credit

*   **(5 points)** Modify the above MinMax strategy to be **4-ply (one Max, one Min, one Max, one Min)**.
*   **(10 points)** Perform **alpha-beta pruning** for the MinMax strategy.

In [208]:
def evl(board):
    piece_scores = {
        PAWN_WHITE: 1,
        PAWN_BLACK: -1,
        ROOK_WHITE: 5,
        ROOK_BLACK: -5,
        # Assign scores for other pieces similarly
    }

    score = 0
    for row in board:
        for piece in row:
            if piece in piece_scores:
                score += piece_scores[piece]

    return score


def GetMinMaxMove(board, player):

    max_eval = float('-inf')  # Initialize max_eval to negative infinity
    best_move = None
    def minmax(board, depth, maximizing_player):
        if depth == 0 or game_over:
            return evl(board), None  # Return evaluation score and move

        legal_moves = GetListOfLegalMoves(board, player if maximizing_player else opponent)
        if maximizing_player:
            max_eval = float('-inf')
            best_move = None
            for move in legal_moves:
                from_square, to_square = move
                # Apply the move temporarily
                piece_at_to_square = board[to_square[0]][to_square[1]]
                board[to_square[0]][to_square[1]] = board[from_square[0]][from_square[1]]
                board[from_square[0]][from_square[1]] = EMPTY

                # Recursively call minmax with depth - 1 and the opposite player
                eval_score, _ = minmax(board, depth - 1, False)

                # Undo the move
                board[from_square[0]][from_square[1]] = board[to_square[0]][to_square[1]]
                board[to_square[0]][to_square[1]] = piece_at_to_square

                if eval_score > max_eval:
                    max_eval = eval_score
                    best_move = move
            return max_eval, best_move
        else:
            min_eval = float('inf')
            best_move = None
            for move in legal_moves:
                from_square, to_square = move
                # Apply the move temporarily
                piece_at_to_square = board[to_square[0]][to_square[1]]
                board[to_square[0]][to_square[1]] = board[from_square[0]][from_square[1]]
                board[from_square[0]][from_square[1]] = EMPTY

                # Recursively call minmax with depth - 1 and the opposite player
                eval_score, _ = minmax(board, depth - 1, True)

                # Undo the move
                board[from_square[0]][from_square[1]] = board[to_square[0]][to_square[1]]
                board[to_square[0]][to_square[1]] = piece_at_to_square

                if eval_score < min_eval:
                    min_eval = eval_score
                    best_move = move
            return min_eval, best_move

    # Get a list of legal moves for the current player
    legal_moves = GetPiecesWithLegalMoves(board, player)

    # Initialize variables for game_over and best_move
    game_over = False
    best_move = None

    opponent = "Black" if player == "White" else "White"

    # Call minmax for each legal move to find the best move
    for move in legal_moves:
        from_square, to_square = move
        # Apply the move temporarily
        piece_at_to_square = board[to_square[0]][to_square[1]]
        board[to_square[0]][to_square[1]] = board[from_square[0]][from_square[1]]
        board[from_square[0]][from_square[1]] = EMPTY

        # Call minmax with depth 2 (2-ply) and the opposite player
        eval_score, _ = minmax(board, 2, False)

        # Undo the move
        board[from_square[0]][from_square[1]] = board[to_square[0]][to_square[1]]
        board[to_square[0]][to_square[1]] = piece_at_to_square

        if eval_score > max_eval:
            max_eval = eval_score
            best_move = move

    return best_move


#Game Setup & Main Loop

(05 points)

Write code below to have a game-play between two AIs - Random vs MinMax. For each iteration, draw the board before and after a turn.

In [209]:
# Define the IsStalemate function
def IsStalemate(board, player):
    # Check if the player has no legal moves but is not in check
    pieces_with_legal_moves = GetPiecesWithLegalMoves(board, player)

    return not bool(pieces_with_legal_moves) and not IsInCheck(board, player)

# Initialize and setup the board
board = ChessBoardSetup()
player = "White"
opponent = "Black"
turns = 0
N = 100  # Maximum number of turns to avoid infinite loops
# Main game loop - while a player is not in checkmate or stalemate (<N turns)
# Below is the rough looping strategy
while not IsCheckmate(board, player) and turns < N:
    DrawBoard(board)

    # Write code to take turns and move the pieces
    if player == "White":
        move = GetRandomMove(board, player)
    else:
        move = GetMinMaxMove(board, player)

    if move:
        from_square, to_square = move
        MovePiece(board, from_square, to_square)

    # Check for checkmate, stalemate, and switch players
    if IsCheckmate(board, player):
        print(f"{player} is in checkmate. {opponent} wins!")
        break
    elif IsStalemate(board, player):
        print("Stalemate! The game is a draw.")
        break

    player, opponent = opponent, player  # Switch players
    turns += 1

# Check and print - Stalemate or Checkmate
if IsCheckmate(board, player):
    print(f"{player} is in checkmate. {opponent} wins!")
elif IsStalemate(board, player):
    print("Stalemate! The game is a draw.")


White is in checkmate. Black wins!


#Submission Instructions

1.   Complete all tasks above - **File MUST contain the output for ALL cells**
2.   Export this notebook as .ipynb
      (File > Download as ipynb)
3.   Upload the .ipynb file on Blackboard



##Rubric

*   (10 points) Chess Board Setup
*   (50 points) Chess Rules Setup
*   (10 points) Random AI
*   (50 points) MinMax AI (2-ply)
*   (05 points) Game Main Loop - Random vs MinMax
*   (15 points) Extra Credit - 4-ply MinMax + alpha-beta pruning



